This notebook demonstrates sentiment analysis using LSTM.

Original source code: https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/2%20-%20Upgraded%20Sentiment%20Analysis.ipynb

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext import data
from torchtext import datasets
import spacy
import itertools
import time
import os

In [2]:
!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


When we use `include_lengths`, this produces a tuple (first element - the text, second element - its length). This allows usage of packed padded sequences.

In [0]:
TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

In [4]:
DATA_PATH = os.path.expanduser(r"~/pytorch_data")
DATA_PATH

'/root/pytorch_data'

In [5]:
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL, root=DATA_PATH)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:08<00:00, 10.2MB/s]


In [6]:
len(train_data), len(test_data)

(25000, 25000)

In [0]:
train_data, valid_data = train_data.split()

In [8]:
len(train_data), len(valid_data), len(test_data)

(17500, 7500, 25000)

In [9]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data,
                 max_size = MAX_VOCAB_SIZE,
                 vectors = "glove.6B.100d",
                 vectors_cache = os.path.join(DATA_PATH, "vector_cache"))

LABEL.build_vocab(train_data)

/root/pytorch_data/vector_cache/glove.6B.zip: 862MB [06:31, 2.20MB/s]                           
100%|█████████▉| 398399/400000 [00:20<00:00, 19400.60it/s]

In [10]:
len(TEXT.vocab)

25002

In [11]:
TEXT.vocab.vectors.shape

torch.Size([25002, 100])

In [12]:
TEXT.vocab.vectors[TEXT.vocab.stoi[TEXT.unk_token]]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])

In [13]:
TEXT.vocab.vectors[TEXT.vocab.stoi[TEXT.pad_token]]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])

In [14]:
', '.join(itertools.islice(TEXT.vocab.stoi.keys(), 100))

'<unk>, <pad>, the, ,, ., and, a, of, to, is, in, I, it, that, ", \'s, this, -, /><br, was, as, with, movie, for, film, The, but, (, n\'t, on, ), you, are, not, have, his, be, he, one, !, by, at, all, an, who, they, from, like, so, her, or, about, has, \', It, out, just, do, ?, some, good, more, very, would, up, what, This, there, time, can, when, which, had, she, if, really, only, story, even, were, their, no, see, my, me, does, did, ..., than, :, much, been, get, could, into, well, people, we, will, other'

In [15]:
uppercase_words = [key for key in TEXT.vocab.stoi.keys() if any([c for c in key if c.isupper()])]
len(uppercase_words), ', '.join(itertools.islice(uppercase_words, 100))

(9043,
 'I, The, It, This, />The, But, And, A, There, He, In, If, />I, They, You, What, TV, She, As, That, DVD, American, />This, When, So, John, We, One, Hollywood, For, However, Not, My, All, />It, THE, No, Even, At, After, Well, His, Do, To, Michael, New, Why, Then, How, While, Of, Also, Some, Mr., James, Just, With, />There, Now, />In, David, English, />If, Robert, British, Man, Paul, Oh, />But, Although, On, Richard, First, OK, Jack, York, King, Lee, George, Oscar, God, Disney, Unfortunately, Maybe, Tom, Yes, Peter, Who, Her, Is, Japanese, />A, From, America, B, French, />And, These, Its, Jane')

In [16]:
LABEL.vocab.stoi

defaultdict(<function torchtext.vocab._default_unk_index>,
            {'neg': 0, 'pos': 1})

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [0]:
BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

In [0]:
class ModelWithLSTM(nn.Module):
    def __init__(self, embeddings, hidden_dim, output_dim, n_layers, bidirectional, dropout, pad_idx):
        super().__init__()
        
        self.embedding = nn.Embedding.from_pretrained(embeddings, padding_idx=pad_idx, freeze=False)
        
        self.rnn = nn.LSTM(embeddings.shape[1], 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        num_directions = 2 if bidirectional else 1
        self.fc = nn.Linear(hidden_dim * num_directions, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, text, text_lengths):
        # text = [sentence len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        # embedded = [sentence len, batch size, emb dim]
        
        # Pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        _, (hidden, _) = self.rnn(packed_embedded)

        # hidden = [num layers * num directions, batch size, hid dim]
        
        # Concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        # and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        # hidden = [batch size, hid dim * num directions]

        return self.fc(hidden)

In [0]:
model = ModelWithLSTM(
    TEXT.vocab.vectors,
    hidden_dim = 256,
    output_dim = 1,
    n_layers = 2, 
    bidirectional = True,
    dropout = 0.5,
    pad_idx = TEXT.vocab.stoi[TEXT.pad_token])

In [21]:
print("Total number of params in model:", sum(p.numel() for p in model.parameters() if p.requires_grad))

Total number of params in model: 4810857


In [22]:
model.embedding.weight.data

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.4098,  1.0487, -0.2304,  ..., -0.6889, -0.8636, -0.6693],
        [ 0.0694,  0.5585,  0.2600,  ...,  0.5310, -0.4120,  0.5282],
        [-0.2574, -0.2177,  0.5419,  ...,  0.3039, -0.8910,  0.4157]])

In [0]:
optimizer = optim.Adam(model.parameters())

In [0]:
model = model.to(device)

criterion = nn.BCEWithLogitsLoss()
criterion = criterion.to(device)

In [0]:
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    return correct.sum() / len(correct)

In [0]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        text.to(device)
        text_lengths.to(device)
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            text, text_lengths = batch.text
            text.to(device)
            text_lengths.to(device)
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [34]:
N_EPOCHS = 5
MODEL_PATH = os.path.join(DATA_PATH, "sentiment_lstm_best.pt")

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), MODEL_PATH)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 1m 38s
	Train Loss: 0.655 | Train Acc: 60.65%
	 Val. Loss: 0.635 |  Val. Acc: 64.70%
Epoch: 02 | Epoch Time: 1m 38s
	Train Loss: 0.575 | Train Acc: 69.66%
	 Val. Loss: 0.503 |  Val. Acc: 77.86%
Epoch: 03 | Epoch Time: 1m 38s
	Train Loss: 0.485 | Train Acc: 77.80%
	 Val. Loss: 0.381 |  Val. Acc: 84.24%
Epoch: 04 | Epoch Time: 1m 38s
	Train Loss: 0.347 | Train Acc: 85.35%
	 Val. Loss: 0.299 |  Val. Acc: 87.24%
Epoch: 05 | Epoch Time: 1m 38s
	Train Loss: 0.234 | Train Acc: 90.90%
	 Val. Loss: 0.279 |  Val. Acc: 88.18%


In [35]:
model.load_state_dict(torch.load(MODEL_PATH))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.320 | Test Acc: 85.66%


In [0]:
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [38]:
predict_sentiment(model, "This film is terrible")

0.0073082586750388145

In [39]:
predict_sentiment(model, "This film is great")

0.9828689098358154